# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

Identifying students as needing intervention or not is a boolean decision with two discrete outputs, not computing a value along a range of coninuous values.

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print("Student data read successfully!")
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [3]:
n_students = student_data.shape[0]
n_features = student_data.shape[1] - 1 #subtract the label column
n_passed = student_data.passed.value_counts()['yes']
n_failed = student_data.passed.value_counts()['no']
grad_rate = 100.*n_passed/n_students
print ("Total number of students: {}".format(n_students))
print ("Number of students who passed: {}".format(n_passed))
print ("Number of students who failed: {}".format(n_failed))
print ("Number of features: {}".format(n_features))
print ("Graduation rate of the class: {:.2f}%".format(grad_rate))

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [4]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print (X_all.head())  # print the first 5 rows
print y_all.head()

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [5]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print ("Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns)))
#y_all = y_all.replace(['yes', 'no'], [1, 0])
#print y_all.shape
#for i,f in enumerate(X_all.columns):
#    print str(i) +": "+X_all.columns[i]

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [64]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

if 0:
    # Stratified Train/Test split
    from sklearn.cross_validation import StratifiedShuffleSplit
    cv = StratifiedShuffleSplit(y_all, n_iter=3, test_size=0.24, random_state=3)
    #print len(cv)
    #print cv

    for train_index, test_index in cv:
        print "TRAIN:", len(train_index), "TEST:", len(test_index)
        # Initial models will use the last training set
        X_train, X_test = X_all.iloc[train_index], X_all.iloc[test_index]
        y_train, y_test = y_all.iloc[train_index], y_all.iloc[test_index] 
else:
    cv = 3
    # Then, select features (X) and corresponding labels (y) for the training and test sets
    # Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
    student_data.iloc[np.random.permutation(len(student_data))]
    student_data.reset_index(drop=True)
    X_train = X_all.iloc[:num_train, :]
    y_train = y_all.iloc[:num_train]
    X_test = X_all.iloc[num_train:, :]
    y_test = y_all.iloc[num_train:]
    print "Training set: {} samples".format(X_train.shape[0])
    print "Test set: {} samples".format(X_test.shape[0])
    # Note: If you need a validation set, extract it from within training data



Training set: 300 samples
Test set: 95 samples


In [21]:
#y_all

## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

### 4.1 K-Nearest Neighbors Classification
K-Nearest neighbors (KNN) is an instance-based learning mechanism that instead of finding an equation to split data will classify a new point based on the points closest to it. A key assumption behind the model is that the entire set of data is not needed to classify a new point. A strength of KNN classification is that processing time for predictions is very fast because only a small number *K* datapoints are required to make a prediction. However, this means that all of the data have to be stored so that any prediction can be made.

In [8]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)
    return end-start

In [50]:
# Choose a model, import it and instantiate an object
from sklearn import neighbors
n_neighbors = 3
weights = 'uniform'
#weights = 'distance'
clf = neighbors.KNeighborsClassifier(weights=weights, n_neighbors=n_neighbors)#, weights=weights)

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
print clf  # you can inspect the learned model by printing it

Training KNeighborsClassifier...
Done!
Training time (secs): 0.002
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')


In [51]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    proc_time = end - start
    print "Done!\nPrediction time (secs): {:.4f}".format(end - start)
    return proc_time, f1_score(target.values, y_pred, pos_label='yes')

In [52]:
train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score[1])
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test)[1])

Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.0073
F1 score for training set: 0.895104895105
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.0029
F1 score for test set: 0.736


In [53]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    t_train = train_classifier(clf, X_train, y_train)
    
    t_train_set, F1_train = predict_labels(clf, X_train, y_train)
    print "F1 score for training set: {}".format(F1_train)
    t_test, F1_test = predict_labels(clf, X_test, y_test)
    print "F1 score for test set: {}".format(F1_test)
    
    return len(X_train), t_train, t_test, F1_train, F1_test
    
     

# Run the helper function above for desired subsets of training data
# Note: Keep the test set constant
training_sizes = [100,200,300]
results = []
for cur_size in training_sizes:
    X1_train, y1_train = X_train[:cur_size], y_train[:cur_size]
    results.append(train_predict(clf, X1_train, y1_train, X_test, y_test))

def print_table(results):
    print
    print "| Training Size | Training Time (secs) | Prediction Time (secs) | F1 Train | F1 Test |"
    print "| --- | --- | --- | --- | --- |"
    for r in results: 
        print "|",
        print str(int(r[0])) +'|'+ str(round(r[1],4))+" |"+str(round(r[2],4))+" |"+str(round(r[3],4))+" |"+str(round(r[4],4))+" |"
        #for i in r: print str(round(i,4))+" |",
        #print
        
print_table(results)

------------------------------------------
Training set size: 100
Training KNeighborsClassifier...
Done!
Training time (secs): 0.001
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.0017
F1 score for training set: 0.898734177215
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.0017
F1 score for test set: 0.645669291339
------------------------------------------
Training set size: 200
Training KNeighborsClassifier...
Done!
Training time (secs): 0.001
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.0041
F1 score for training set: 0.881355932203
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.0022
F1 score for test set: 0.732824427481
------------------------------------------
Training set size: 300
Training KNeighborsClassifier...
Done!
Training time (secs): 0.001
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.0065
F1 score for tra

### 4.2 Decision Tree Classification

In [29]:
# Choose a model, import it and instantiate an object
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion='entropy', min_samples_leaf=8)#, max_features=7)#, max_depth=4)
    #max_leaf_nodes=8)
# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
print clf  # you can inspect the learned model by printing it

# Predict on training set and compute F1 score
train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)[1]

# Predict on test data
print
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test)[1])

# Run the helper function above for desired subsets of training data
# Note: Keep the test set constant
training_sizes = [100,200,300]
results = []
for cur_size in training_sizes:
    X1_train, y1_train = X_train[:cur_size], y_train[:cur_size]
    results.append(train_predict(clf, X1_train, y1_train, X_test, y_test))

print_table(results)

Training DecisionTreeClassifier...
Done!
Training time (secs): 0.002
DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=8,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.0003
1

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.0002
F1 score for test set: 0.64347826087
------------------------------------------
Training set size: 100
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.0002
F1 score for training set: 0.844444444444
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.0002
F1 score for test set: 0.666666666667
------------------------------------------
Training

### 4.3 Support Vector Machines
computation increases rapidly with number of training vectors. Although may not be an issue when train size is constrained.

In [30]:
# Choose a model, import it and instantiate an object
from sklearn.svm import SVC
clf = SVC()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
print clf  # you can inspect the learned model by printing it

# Predict on training set and compute F1 score
train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score[1])

# Predict on test data
print
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test)[1])

# Run the helper function above for desired subsets of training data
# Note: Keep the test set constant
training_sizes = [100,200,300]
results = []
for cur_size in training_sizes:
    X1_train, y1_train = X_train[:cur_size], y_train[:cur_size]
    results.append(train_predict(clf, X1_train, y1_train, X_test, y_test))

print_table(results)

Training SVC...
Done!
Training time (secs): 0.007
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Predicting labels using SVC...
Done!
Prediction time (secs): 0.0049
F1 score for training set: 0.854700854701

Predicting labels using SVC...
Done!
Prediction time (secs): 0.0016
F1 score for test set: 0.826666666667
------------------------------------------
Training set size: 100
Training SVC...
Done!
Training time (secs): 0.001
Predicting labels using SVC...
Done!
Prediction time (secs): 0.0008
F1 score for training set: 0.819277108434
Predicting labels using SVC...
Done!
Prediction time (secs): 0.0008
F1 score for test set: 0.805031446541
------------------------------------------
Training set size: 200
Training SVC...
Done!
Training time (secs): 0.003
Predicting labels using SVC...
Done!
Prediction time (secs): 

# Tuning Area

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

Explain the best model. 
Resources
Cost 
Performance

Describe in layman's terms how it will work


In [65]:
# Tune the best model
# Choose a model, import it and instantiate an object
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

dt_clf = DecisionTreeClassifier(
    criterion='entropy',
    random_state=None
)

parameters = {'max_depth': [2,3,4,5,6],
             'min_samples_leaf': [5,8,10,15]}
    
dt_score = make_scorer(f1_score, pos_label='yes')

clf = GridSearchCV(
    dt_clf, 
    parameters, 
    scoring=dt_score, 
    cv=cv)


#clf.fit(X_train, y_train)
train_classifier(clf, X_train, y_train)
print clf
print "Best estimator"
print clf.best_estimator_
print clf.best_params_
print clf.best_score_

for i,f in enumerate(clf.best_estimator_.feature_importances_):
    if f > 0:
        print X_all.columns[i], f

print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test)[1])


Training GridSearchCV...
Done!
Training time (secs): 0.281
GridSearchCV(cv=3, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_depth': [2, 3, 4, 5, 6], 'min_samples_leaf': [5, 8, 10, 15]},
       pre_dispatch='2*n_jobs', refit=True,
       scoring=make_scorer(f1_score, pos_label=yes), verbose=0)
Best estimator
DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=2,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=8,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
{'max_depth': 2, 'min_samples_leaf': 8}
0.815404081193
age 0.119258214868
failure

In [68]:
# Tune the best model
# Choose a model, import it and instantiate an object
#from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import make_scorer, f1_score

clf = SVC()

parameters = {'C': [.92,.94,.96,.98,1.]}
    
grid_scorer = make_scorer(f1_score, pos_label='yes')

grid = GridSearchCV(
    clf, 
    parameters, 
    scoring=grid_scorer, 
    cv=cv)


#clf.fit(X_train, y_train)
train_classifier(grid, X_train, y_train)
print grid
print "Best estimator"
print grid.best_estimator_
print grid.best_params_
print grid.best_score_

#for i,f in enumerate(grid.best_estimator_.feature_importances_):
#    if f > 0:
#        print X_all.columns[i], f

print "F1 score for test set: {}".format(predict_labels(grid, X_test, y_test)[1])


Training GridSearchCV...
Done!
Training time (secs): 0.137
GridSearchCV(cv=3, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [0.92, 0.94, 0.96, 0.98, 1.0]},
       pre_dispatch='2*n_jobs', refit=True,
       scoring=make_scorer(f1_score, pos_label=yes), verbose=0)
Best estimator
SVC(C=0.94, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
{'C': 0.94}
0.813006124882
Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 0.0017
F1 score for test set: 0.757142857143


SVC looked promising in the 100 sample training size condition. However, it doesn't perform well with tuning.